- This computation for the Value Function works if the state space is not too large.
- Whe the state space is large, this direct method of solving a linear system of equations will not scale.
- We need to resort to numerical methods (DP, RL)


In [ ]:
MarkovRewProcessDict = {"Current State A":{"Action 1":{"NextS1fromA": ("PNextS1fromA","Reward1")
                                           ,"NextS2fromA, from A": ("PNextS2fromA","Reward2")},
                                           "Action 2":{"NextS1fromA": ("PNextS1fromA","Reward1")
                                           ,"NextS2fromA, from A": ("PNextS2fromA","Reward2")}},
                    
                     "Current State B":{"NextS1fromB": ("PNextS1fromB","Reward3"),
                                        "NextS2fromB": ("PNextS2fromB","Reward4")}}

MarkovRewProcessDict
MarkovRewProcessDict["Current State A"]['Action 1']["NextS1fromA"]
#MarkovRewProcessDict: Dict[tuple, Dict[tuple, tuple]] = {}

MDP_dict: Dict[tuple, Dict[tuple, tuple]] = {}
user_capacity = 2
user_poisson_lambda = 1.0

holding_cost = 1
stockout_cost = 10

gamma = 0.9 
MDP_dict: Dict[tuple, Dict[tuple, tuple]] = {}
user_cap = 2
poisson_lambda = 1.0

holding_cost = 1
stockout_cost = 10

gamma = 0.9

MDP_dict: Dict[tuple, Dict[tuple, tuple]] = {}

for alpha in range(user_cap+1):                            
                                                               
    for beta in range(user_cap + 1 - alpha):
        
        # This is St, the current state
        state = (alpha, beta)                                   

        # This is initial inventory, total bike you have at 8AM 
        init_inv = alpha + beta                         
        
        # The beta1 is the beta in next state, irrespctive of current state (as the decsion policy is constant)
        beta1 = user_cap - init_inv
        
        base_reward = -alpha* holding_cost
        # List of all possible demand you can get
        
        #dict1 = {}
        action = {}

        for order in range(user_cap-init_inv +1):
            
            #action = {}
            dict1 = {}
            for i in range(init_inv +1):

            # if initial demand can meet the deman
                if i <= (init_inv-1):
                
                # probality of specifc demand can happen
                    transition_prob = poisson.pmf(i,poisson_lambda)

                    dict1[((init_inv - i, order), base_reward)] = transition_prob

                         
            # if initial demand can not meet the demand
                else:
                
                    transition_prob = 1- poisson.cdf(init_inv -1, poisson_lambda)
                
                # probability of not meeting the demands
                    transition_prob2 = 1- poisson.cdf(init_inv, poisson_lambda)
                
                # total reward
                
                    reward = base_reward - stockout_cost*((poisson_lambda*transition_prob) - \
                                                  init_inv*transition_prob2)                

                    dict1[((init_inv - i, order),reward)] = transition_prob

                    #if state in MDP_dict:

            action[order] = dict1

        MDP_dict[state]= action
MDP_dict
MDP_dict
from typing import Dict, Tuple
from scipy.stats import poisson

# Constants
user_cap = 2
poisson_lambda = 1.0
holding_cost = 1
stockout_cost = 10
gamma = 0.9

# Initialize the MDP dictionary
MDP_dict: Dict[Tuple, Dict[Tuple, float]] = {}

# Loop through possible values of alpha and beta
for alpha in range(user_cap + 1):
    for beta in range(user_cap + 1 - alpha):
        state = (alpha, beta)
        init_inv = alpha + beta
        #beta1 = user_cap - init_inv
        base_reward = -alpha * holding_cost

        action = {}
        for order in range(user_cap - init_inv + 1):
            dict1 = {}
            for i in range(init_inv + 1):
                if i <= (init_inv - 1):
                    transition_prob = poisson.pmf(i, poisson_lambda)
                    dict1[((init_inv - i, order), base_reward)] = transition_prob
                else:
                    transition_prob = 1 - poisson.cdf(init_inv - 1, poisson_lambda)
                    transition_prob2 = 1 - poisson.cdf(init_inv, poisson_lambda)
                    reward = base_reward - stockout_cost * ((poisson_lambda * transition_prob) -
                                                            init_inv * transition_prob2)
                    dict1[((init_inv - i, order), reward)] = transition_prob
            action[order] = dict1
        MDP_dict[state] = action

MDP_dict